In [439]:
import pandas as pd
import numpy as np
import os
import json
import argparse
import getpass
import random

# from pymongo import MongoClient
from bs4 import BeautifulSoup

from json import loads

import nltk
from nltk import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer


In [7]:
DATALOC = '/Users/annalie/Dropbox/Pending/Dev/refugees-hackathon/social_listening/'

KEYWORDS = ['refugee', 'refugees', 'migrant', 'migrants', 'asylum', 'rohingya',
            'immigrant', 'immigrants', 'UNHCR', 'UN Refugees', 'ICE',
            'deportation', 'border wall', 'illegal border crossing', 'syrians',
           'rohingya', 'fleeing']

In [8]:
def get_jsonfiles(foldername):
    '''Get all files from folder'''
    foldername = os.path.join(DATALOC, foldername)
    return [jsonfile for jsonfile in os.listdir(foldername) if jsonfile.endswith('.json')]

In [9]:
foldername = 'freedomworks/'
get_jsonfiles(foldername)

['freedomworks0.json',
 'freedomworks1.json',
 'freedomworks2.json',
 'freedomworks3.json',
 'freedomworks4.json',
 'freedomworks5.json']

In [236]:
def get_article_gen(path):
    """ Generator that yields one article at a time. """
    with open(path,'r') as file:
        for line in file:
            yield loads(line.strip('\n'))
            
# FILEPATH = './nytimes52.json'
FILEPATH = '/Users/annalie/Dropbox/Pending/Dev/refugees-hackathon/social_listening/freedomworks/freedomworks0.json'

# create a generator that yields articles from the file
articles = get_article_gen(FILEPATH)

In [12]:
def ReadContent():
    p = soup.find('div', attrs = {'class': 'field-item even'})
    t = p.text
    return unicode(t.encode('utf-8'), errors='ignore')

In [24]:
def ReadTitle():
    p = soup.find('h1')
    t = p.text
    return unicode(t.encode('utf-8'), errors='ignore')

In [237]:
text = []
title = []

In [291]:
# get one article and take a look at it
article = next(articles)
soup = BeautifulSoup(article['html'])
title.append(ReadTitle())
title[-1]

u'Government Protects You From... Small Grapefruits?'

In [286]:
text.append(ReadContent())

In [292]:
text[-1]

u'WHAT: Former Virginia attorney general Ken Cuccinelli, along with members of the FreedomWorks Activist Advisory Council, will host a press call to kick-off a nationwide Digital Day of Action to defend internet freedom.\nWHEN: Tuesday, May 8, 2018 at 10:00 AM ET.\nWHERE: Contact freedomworks@pinkstongroup.com for the call-in details.\nWHO: Participants on the call will include:\nKen Cuccinelli, director of FreedomWorks Regulatory Action Center and former attorney general of Virginia\nPatrick Hedger, director of policy for FreedomWorks\nDavid Zupan, Ohio activist and president of the FreedomWorks Activist Advisory Council \nJohn Mcavoy, Ohio activist and president of Toledo Tea Party\nRonda Vuillemont-Smith, Oklahoma activist and president of Tulsa 9.12 Project \nJack Adkins, Oregon activist \nCarol Davis, Illinois activist and founder of the West Suburban Patriots\nWHY: Silicon Valley special interests are holding an online demonstration on May 9 to pressure members of Congress to use

In [295]:
new = []
for i in text:
    new.append(i.replace('\n', ' '))

In [300]:
text = list(set(new))

## Sentiment analyses on text

#### Bigrams

In [424]:
def pairs(text, tag1='', tag2=''):
    tagged_pair = nltk.bigrams(nltk.pos_tag(word_tokenize(text), tagset='universal'))
    pair = set([(a[0], b[0]) for (a,b) in tagged_pair if ((a[1]==tag1) and (b[1]==tag2))])
    return pair

In [419]:
pairs(text[-1], 'ADJ', 'NOUN')

{(u'coming', u'months'),
 (u'long', u'way'),
 (u'more', u'waste'),
 (u'national', u'debt'),
 (u'other', u'tools'),
 (u'past', u'years'),
 (u'right', u'direction'),
 (u'unobligated', u'budget')}

In [318]:
text[-1]

u'Washington, D.C.- Today, President Trump transmitted to Congress his proposal to rescind $15.4 billion in unobligated budget authorities from past years, as allowed for under the Congressional Budget and Impoundment Control Act of 1974. This tool was used by every president, every year, from 1974 through 2000, but has not been drawn on in over 17 years. FreedomWorks President Adam Brandon commented: FreedomWorks applauds the work of the White House and President Trump in transmitting to Congress $15.4 billion in proposed rescissions. This is a step in the right direction towards cutting spending, as Republicans promise year after year to do. But, with a national debt of over $21 trillion, there is a long way to go. "Congress should act swiftly in approving this proposal in full, as one step in an effort that does not stop there. They should work with the administration in the coming months and years to use this revived rescissions tool and other tools available to them to cut more wa

#### Trigrams

In [385]:
def pairs(text, tag1=None, tag2=None, tag3=None):
    tn = []
    tagged_pair = nltk.trigrams(nltk.pos_tag(word_tokenize(text[-1]), tagset='universal'))
    if tag3 is not None:
        t = list(set([(a[0], b[0], c[0]) for (a,b,c) in tagged_pair if (a[1]==tag1) and (b[1]==tag2) and (c[1]==tag3)]))
    if tag3 == None:
        t = list(set([(a[0], b[0], c[0]) for (a,b,c) in tagged_pair if (a[1]==tag1) and (b[1]==tag2)]))        
    for i in t:
        tn.append(' '.join(i))
    return tn

In [393]:
phrases = []
phrases.extend(pairs(text, tag1='VERB', tag2='DET', tag3='NOUN'))
phrases.extend(pairs(text, tag1='VERB', tag2='VERB'))
phrases.extend(pairs(text, tag1='NOUN', tag2='VERB'))

In [394]:
phrases

[u'is a step',
 u'approving this proposal',
 u'applauds the work',
 u'should act swiftly',
 u'should work with',
 u'been drawn on',
 u'was used by',
 u'towards cutting spending',
 u'FreedomWorks applauds the',
 u'Trump transmitted to',
 u'tool was used',
 u'Brandon commented :',
 u'Congress should act',
 u'Republicans promise year']

In [ ]:
sid = SentimentIntensityAnalyzer()

In [400]:
sentiment = []
for i in phrases:
    sentiment.append(sid.polarity_scores(i)['compound'])

In [402]:
zip(phrases, sentiment)

[(u'is a step', 0.0),
 (u'approving this proposal', 0.0),
 (u'applauds the work', 0.34),
 (u'should act swiftly', 0.296),
 (u'should work with', 0.0),
 (u'been drawn on', 0.0),
 (u'was used by', 0.0),
 (u'towards cutting spending', -0.128),
 (u'FreedomWorks applauds the', 0.34),
 (u'Trump transmitted to', 0.0),
 (u'tool was used', 0.0),
 (u'Brandon commented :', 0.0),
 (u'Congress should act', 0.0),
 (u'Republicans promise year', 0.3182)]

In [417]:
np.mean([i for i in sentiment if i !=0])

0.23324000000000003

## Keyword extraction from text

In [444]:
# split into words
tokens = word_tokenize(text[-1])
# convert to lower case
tokens = [w.lower() for w in tokens]
# remove remaining tokens that are not alphabetic
words = [word for word in tokens if word.isalpha()]
# filter out stop words
stop_words = set(stopwords.words('english'))
words = [w for w in words if not w in stop_words]
# lemmatize verbs
words = [wnl.lemmatize(i, pos='v') for i in words]

In [445]:
print(words)

[u'washington', u'today', u'president', u'trump', u'transmit', u'congress', u'proposal', u'rescind', u'billion', u'unobligated', u'budget', u'authorities', u'past', u'years', u'allow', u'congressional', u'budget', u'impoundment', u'control', u'act', u'tool', u'use', u'every', u'president', u'every', u'year', u'draw', u'years', u'freedomworks', u'president', u'adam', u'brandon', u'comment', u'freedomworks', u'applaud', u'work', u'white', u'house', u'president', u'trump', u'transmit', u'congress', u'billion', u'propose', u'rescissions', u'step', u'right', u'direction', u'towards', u'cut', u'spend', u'republicans', u'promise', u'year', u'year', u'national', u'debt', u'trillion', u'long', u'way', u'go', u'congress', u'act', u'swiftly', u'approve', u'proposal', u'full', u'one', u'step', u'effort', u'stop', u'work', u'administration', u'come', u'months', u'years', u'use', u'revive', u'rescissions', u'tool', u'tool', u'available', u'cut', u'waste', u'include', u'spend', u'years', u'irresponsi